# Dependencies

For ZnTrack there are two different ways to set up dependencies:

1. Node based dependencies
2. File based dependencies

## Node dependencies
We will first look at Node based dependencies starting from a RandomNumber `Hello World` example.
In our first stage we create a random number and then we add another Node that depends on this one.
We can do this very easily by using `zn.deps`.

This allows us to access all properties of the `dependency` attribute.

In [1]:
from zntrack import Node, dvc, zn, config
from random import randrange
from pathlib import Path

In [2]:
config.nb_name = "03_dependencies.ipynb"

In [3]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [4]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmpnpwc7foi/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [5]:
class RandomNumber(Node):
    maximum = zn.params()
    number = zn.outs()

    def run(self):
        self.number = float(randrange(self.maximum))


class ComputePower(Node):
    random_number: RandomNumber = zn.deps()
    number = zn.outs()
    power = zn.params()

    def run(self):
        self.number = self.random_number.number**self.power

We can now create the stages the usual way and look at the outcomes.
This will create the following graph for us:


[![](https://mermaid.ink/img/pako:eNpVjLEKwkAQRH8lTG0KLa-wMbWI2rkWa25jAtm7cO4RJOTfPQQLu2Hem1nQRi9w6MY4tz0nq64NBbJQ6u2NcIg6ZZNTnCUR7j-0K-jMwUc9Zn38o7ref9fYQCUpD778LxSqimC9qBBciV46zqMRKKxF5Wzx8g4tnKUsG-TJs0kz8DOxwnU8vmT9ACo6PUg?type=png)](https://mermaid.live/edit#pako:eNpVjLEKwkAQRH8lTG0KLa-wMbWI2rkWa25jAtm7cO4RJOTfPQQLu2Hem1nQRi9w6MY4tz0nq64NBbJQ6u2NcIg6ZZNTnCUR7j-0K-jMwUc9Zn38o7ref9fYQCUpD778LxSqimC9qBBciV46zqMRKKxF5Wzx8g4tnKUsG-TJs0kz8DOxwnU8vmT9ACo6PUg)

<div class="alert alert-info">
Note

Instead of passing the class as an argument it is also possible to define a default dependency.

```python
dependency: Stage = zn.deps(Stage) # or zn.deps(Stage.load())
```
</div>

In [6]:
random_number = RandomNumber(maximum=16)
random_number.write_graph()
ComputePower(random_number=random_number, power=2.0).write_graph()

2023-02-16 10:19:16,830 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook 03_dependencies.ipynb to script
/data/fzills/miniconda3/envs/zntrack/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


2023-02-16 10:19:18,182 (WARNING): Running DVC command: 'stage add -n RandomNumber --force ...'


[NbConvertApp] Writing 6156 bytes to 03_dependencies.py


2023-02-16 10:19:19,555 (INFO): 


[NbConvertApp] Converting notebook 03_dependencies.ipynb to script
/data/fzills/miniconda3/envs/zntrack/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


2023-02-16 10:19:20,813 (WARNING): Running DVC command: 'stage add -n ComputePower --force ...'


[NbConvertApp] Writing 6156 bytes to 03_dependencies.py


2023-02-16 10:19:22,080 (INFO): 


In [7]:
!dvc repro

Running stage 'RandomNumber':                                         
> zntrack run src.RandomNumber.RandomNumber --name=RandomNumber
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

Running stage 'ComputePower':
> zntrack run src.ComputePower.ComputePower --name=ComputePower
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


In [8]:
print(
    f"{RandomNumber.load().number} ^ {ComputePower.load().power} ="
    f" {ComputePower.load().number}"
)

1.0 ^ 2.0 = 1.0


## File dependencies
The second approach for specifying dependencies in ZnTrack is to depend on files.
This is useful when our pipeline requires output files from a previous stage, or when we want to track the changes in an input file.
To create a file dependency, we first create a file from our random number.
We then use the path to that file as our dependency.
Setting a file dependency is simple and can be done by passing ``pathlib.Path`` or ``str`` to the ``dvc.deps`` method.
Like other ``dvc.<...>`` attributes, it also supports lists:
```py
dependency: Path = dvc.deps([Path('some_file.txt'), 'some_other_file.txt'])
```

<div class="alert alert-info">
Info: Node working directory

It is recommended to store files created by a node in the node's working directory (nwd), which is located at ``./nodes/<nodename>``.
You can access the nwd using ``zntrack.nwd``. Here's an example:

```python
file: Path = dvc.outs(zntrack.nwd / "random_number.txt")
```
</div>


In [9]:
# zntrack: break
from zntrack import nwd

class WriteToFile(Node):
    random_number: RandomNumber = zn.deps(RandomNumber)
    file: Path = dvc.outs(nwd / "random_number.txt")

    def run(self):
        self.file.write_text(str(self.random_number.number))


class PowerFromFile(Node):
    file: Path = zn.deps()
    number = zn.outs()
    power = zn.params(2)

    def run(self):
        number = float(self.file.read_text())
        self.number = number**self.power


class ComparePowers(Node):
    power_deps = zn.deps()

    def run(self):
        assert self.power_deps[0].number == self.power_deps[1].number

Let us create the stages and look at the graph.

In [10]:
write_to_file = WriteToFile()
write_to_file.write_graph()
power_from_file = PowerFromFile(file=write_to_file.file)
power_from_file.write_graph()
ComparePowers(power_deps=[power_from_file, ComputePower.load()]).write_graph()

[NbConvertApp] Converting notebook 03_dependencies.ipynb to script
/data/fzills/miniconda3/envs/zntrack/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


2023-02-16 10:19:25,194 (WARNING): Running DVC command: 'stage add -n WriteToFile --force ...'


[NbConvertApp] Writing 6156 bytes to 03_dependencies.py


2023-02-16 10:19:26,305 (INFO): 


[NbConvertApp] Converting notebook 03_dependencies.ipynb to script
/data/fzills/miniconda3/envs/zntrack/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


...

2023-02-16 10:19:28,975 (INFO): 


[NbConvertApp] Converting notebook 03_dependencies.ipynb to script
/data/fzills/miniconda3/envs/zntrack/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


2023-02-16 10:19:30,262 (WARNING): Running DVC command: 'stage add -n ComparePowers --force ...'


[NbConvertApp] Writing 6156 bytes to 03_dependencies.py


2023-02-16 10:19:31,524 (INFO): 


In [11]:
!dvc dag

                +--------------+                  
                | RandomNumber |                  
                +--------------+                  
                **             ***                
             ***                  ***             
           **                        **           
 +-------------+                       **         
 | WriteToFile |                        *         
 +-------------+                        *         
        *                               *         
        *                               *         
        *                               *         
+---------------+               +--------------+  
| PowerFromFile |               | ComputePower |  
+---------------+               +--------------+  
                **             ***                
                  ***        **                   
                     **    **                     
                +---------------+                 
                | ComparePowers

In [12]:
!dvc repro

Stage 'RandomNumber' didn't change, skipping                          
Stage 'ComputePower' didn't change, skipping
Running stage 'WriteToFile':
> zntrack run src.WriteToFile.WriteToFile --name=WriteToFile
Updating lock file 'dvc.lock'                                                   

Running stage 'PowerFromFile':
> zntrack run src.PowerFromFile.PowerFromFile --name=PowerFromFile
Updating lock file 'dvc.lock'                                                   

Running stage 'ComparePowers':
> zntrack run src.ComparePowers.ComparePowers --name=ComparePowers
Updating lock file 'dvc.lock'                                         

To track the changes with git, run:

	git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


In [13]:
# to verify we can also run the method manually
ComparePowers.load().run()

If we now look at our `dvc.yaml` we can see that for our Node dependencies we rely on the `nodes/<node_name>/outs.json` while for the file dependency it is directly connect to the passed file.

In [14]:
from IPython.display import Pretty, display

display(Pretty("dvc.yaml"))

stages:
  RandomNumber:
    cmd: zntrack run src.RandomNumber.RandomNumber --name=RandomNumber
    deps:
    - src/RandomNumber.py
    params:
    - RandomNumber
    outs:
    - nodes/RandomNumber/outs.json
  ComputePower:
    cmd: zntrack run src.ComputePower.ComputePower --name=ComputePower
    deps:
    - nodes/RandomNumber/outs.json
    - src/ComputePower.py
    params:
    - ComputePower
    outs:
    - nodes/ComputePower/outs.json
  WriteToFile:
    cmd: zntrack run src.WriteToFile.WriteToFile --name=WriteToFile
    deps:
    - nodes/RandomNumber/outs.json
    - src/WriteToFile.py
    outs:
    - nodes/WriteToFile/random_number.txt
  PowerFromFile:
    cmd: zntrack run src.PowerFromFile.PowerFromFile --name=PowerFromFile
    deps:
    - nodes/WriteToFile/random_number.txt
    - src/PowerFromFile.py
    params:
    - PowerFromFile
    outs:
    - nodes/PowerFromFile/outs.json
  ComparePowers:
    cmd: zntrack run src.ComparePowers.ComparePowers --name=ComparePowers
    deps:
    -

## Node attributes as dependencies

It is also possible to specify a Node attribute as a dependency. In this case you will be able to access the value of the attribute directly instead of using the Node class.
This can be used for all `dvc.<option>` and `zn.<option>` as well as e.g. class properties.
Note that the dvc dependencies will still be written for the full Node and won't be limited to the Node attribute.
To be able to define a dependency of an attribute the `zntrack.getdeps` function is required.

In [15]:
from zntrack import getdeps

In [16]:
class ComputePowerFromNumber(Node):
    number: float = zn.deps()  # this will be a float instead of RandomNumber

    power: int = zn.params()
    result: float = zn.outs()

    def run(self):
        self.result = self.number**self.power

In [17]:
ComputePowerFromNumber(number=getdeps(RandomNumber, "number"), power=2.0).write_graph(
    run=True
)

[NbConvertApp] Converting notebook 03_dependencies.ipynb to script
/data/fzills/miniconda3/envs/zntrack/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


2023-02-16 10:19:36,501 (WARNING): Running DVC command: 'stage add -n ComputePowerFromNumber --force ...'


[NbConvertApp] Writing 6156 bytes to 03_dependencies.py


2023-02-16 10:19:37,737 (INFO): 
2023-02-16 10:19:37,738 (WARNING): Running DVC command: 'repro ComputePowerFromNumber'
2023-02-16 10:19:39,767 (INFO): 


`getdeps(RandomNumber, "number")` can also be replaced by `getdeps(RandomNumber["nodename"], "number")` or `getdeps(RandomNumber.load(name="nodename"), "number")`.
The first argument represents the Node and the second argument is the attribute, similar to `getattr()`. ZnTrack also provides a shorthand for this via `RandomNumber @ "number"` or `RandomNumber["nodename"] @ "number"`.

In [18]:
compute_power = ComputePowerFromNumber.load()

In [19]:
print(f"{compute_power.number} ^ {compute_power.power} = {compute_power.result}")

1.0 ^ 2.0 = 1.0


In [20]:
temp_dir.cleanup()